In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df_new = pd.read_csv('../input/av-janatahack-healthcare-hackathon-ii/Data/train.csv')
test_df_new = pd.read_csv('../input/av-janatahack-healthcare-hackathon-ii/Data/test.csv')

In [ ]:
train_df_new['Bill_per_patient'] = train_df_new.groupby('patientid')['Admission_Deposit'].transform('sum')
test_df_new['Bill_per_patient'] = test_df_new.groupby('patientid')['Admission_Deposit'].transform('sum')

In [ ]:
from fastai.tabular import *

In [ ]:
procs = [FillMissing, Categorify]
cat_names = ['Age', 'Severity of Illness', 'Type of Admission', 'Visitors with Patient' ,'City_Code_Patient','Bed Grade','Ward_Facility_Code','Ward_Type','Department','Hospital_region_code','City_Code_Hospital','Hospital_type_code','Hospital_code']
cont_names = ['Admission_Deposit','Bill_per_patient','Available Extra Rooms in Hospital']
dep_var = 'Stay'

data = (TabularList.from_df(train_df_new, procs=procs, cont_names=cont_names, cat_names=cat_names)
        .split_by_idx(valid_idx=range(int(len(train_df_new)*0.99),len(train_df_new)))
        .label_from_df(cols=dep_var)
        .add_test(TabularList.from_df(test_df_new, cat_names=cat_names, cont_names=cont_names, procs=procs))
        .databunch())
print(data.train_ds.cont_names)
print(data.train_ds.cat_names)

In [ ]:
learn = tabular_learner(data, layers=[200,100,50], metrics=accuracy)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:

learn.fit_one_cycle(7)

In [ ]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)
pred_prob, pred_class = preds.max(1)

In [ ]:
pd.DataFrame(pred_class).to_csv("predictions.csv")
output = pd.DataFrame(test_df_new['case_id'].values,columns=['case_id'])
prediction = pd.read_csv('./predictions.csv',names=['Stay'],header=0)


In [ ]:
prediction.loc[prediction['Stay'] == 0, 'Stay'] = '0-10'
prediction.loc[prediction['Stay'] ==1 , 'Stay'] = '11-20'
prediction.loc[prediction['Stay'] ==2 , 'Stay'] = '21-30'
prediction.loc[prediction['Stay'] ==3 , 'Stay'] = '31-40'
prediction.loc[prediction['Stay'] ==4 , 'Stay'] = '41-50'
prediction.loc[prediction['Stay'] ==5 , 'Stay'] = '51-60'
prediction.loc[prediction['Stay'] ==6 , 'Stay'] = '61-70'
prediction.loc[prediction['Stay'] ==7 , 'Stay'] = '71-80'
prediction.loc[prediction['Stay'] ==8 , 'Stay'] = '81-90'
prediction.loc[prediction['Stay'] ==9 , 'Stay'] = '91-100'
prediction.loc[prediction['Stay'] ==10, 'Stay'] = 'More than 100 Days'

In [ ]:
output = pd.DataFrame(test_df_new['case_id'].values,columns=['case_id'])
output['Stay'] = prediction

In [ ]:
output.to_csv('Healthcare_Submission7.csv',index=False)